### Modelos
- Modelo Óbito-Curado (60/40): Regressão
- Modelo Internado-Curado (70/30): AdaBoost

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import import_ipynb
import modelofuncoes as mf

from joblib import dump, load

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import metrics
from sklearn.metrics import accuracy_score

importing Jupyter notebook from modelofuncoes.ipynb


### Avaliando o Smote

In [ ]:
from imblearn.over_sampling import RandomOverSampler, SMOTE

def aplicarSmote(base, alvo):

    X_smt = base.drop(alvo, axis = 1)
    y_smt = base[alvo]
    
    # Contar o número de ocorrências
    # np.bincount(y_smt)
    
    # Carregar o Smote
    smt = SMOTE()
    
    # Aplicar o Smote
    X_smt, y_smt = smt.fit_resample(X_smt, y_smt)
    
    # Contar o número de ocorrências
    # np.bincount(y_smt)
    
    # Convertendo em DF
    smoteBase = pd.DataFrame(X_smt)
    smoteBase[alvo] = y_smt
    
    return smoteBase

### Modelo OC - Constatando o Desempenho

In [2]:
# Carregando a Base de Dados

#url = 'Vacinas/OC/oc-vacina-6040-sp.xlsx'
url = 'PreProcessamento/balanceadas/OC/ba-oc-6040-sp.xlsx'
alvo = 'evolucaoCaso'
baseObitoCurado = pd.read_excel(url)

bases = {}
bases['Base OC (60/40)'] = url

In [ ]:
#baseObitoCurado[alvo].value_counts()

In [3]:
# Definindo o Modelo de Regressão Logística
hiperparametrosLR = LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True, 
                                       intercept_scaling=1, l1_ratio=None, max_iter=1000, multi_class='auto', 
                                       n_jobs=None, penalty='l2', random_state=1340, solver='lbfgs', 
                                       tol=0.0001, verbose=0, warm_start=False)

modeloClassificador = {"LR": hiperparametrosLR}

In [4]:
valoresAlvo = [0, 1]

#colunasRemovidas = ['dataPrimeiraDose', 'dataSegundaDose', 'assintomatico', 'puerpera', 'disturbiosGustatorios', 
#                    'disturbiosOlfativos', 'gestante', 'tosse', 'respiratoria', 'fragilidadeImuno', 'racaCor', 
#                    'sexo', 'febre', 'renal', 'imunossupressao', 'obesidade']

colunasRemovidas = ['assintomatico', 'asma', 'cefaleia', 'tosse', 'diarreia', 
                    'dorNoPeito', 'hipotireoidismo', 'imunossupressao', 'puerpera', 
                    'disturbiosGustatorios', 'disturbiosOlfativos', 'bronquite', 
                    'dosNasCostas', 'gestante', 'respiratoria', 'sexo', 'febre', 
                    'mialgia', 'cardiaca', 'hipertensao',  
                    'faltaDeAr', 'racaCor', 'congestaoNasal', 'renal', 'dorNoCorpo', 
                    'vomito', 'fragilidadeImuno', 'cansaco']



In [48]:
# Verificando o Desempenho do Modelo
desempenhoDosModelos = mf.testarModelos(bases, alvo, colunasRemovidas, modeloClassificador, valoresAlvo)
desempenhoDosModelos

Index(['evolucaoCaso', 'dorDeGarganta', 'dispneia', 'coriza', 'dorDeCabeca',
       'diabetes', 'obesidade', 'qntVacinas', 'faixaetaria',
       'faixaDiasSintomas'],
      dtype='object')


,Base,Modelo,Acurácia,AUC,Precisão (0),Recall (0),F1-Score (0),Precisão (1),Recall (1),F1-Score (1),3 Kfolds,5 Kfolds
0,Base OC (60/40),LR,81.01,0.86,82.43,70.11,75.77,80.2,89.01,84.38,79.79,80.35


In [49]:
# Identificando a média 
# media = desempenhoDosModelos.mean()

# Inserindo as outras colunas
# media['Base'] = 'Média dos Modelos'
# media['Modelo'] = "-"

# Adicionando a média dos valores
# desempenhoDosModelos = desempenhoDosModelos.append(media, ignore_index=True)
# desempenhoDosModelos

# Convertendo series para dict
# media_dict = pd.Series(media.values, index=media.keys).to_dict()

/tmp/ipykernel_7561/1827519719.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  media = desempenhoDosModelos.mean()


### Definindo a Função da Curva ROC

In [ ]:
def exibirCurvaROC(modeloClassificador, x_test, y_test):
    y_preds = modeloClassificador.predict_proba(x_test)

    # take the second column because the classifier outputs scores for the 0 class as well
    preds = y_preds[:,1]

    # fpr (false-positive-rate) e tpr (true-positive-rate)
    fpr, tpr, _ = metrics.roc_curve(y_test, preds)

    auc_score = metrics.auc(fpr, tpr)

    # Construindo o Gráfico
    plt.clf()
    plt.title('ROC Curve')
    plt.plot(fpr, tpr, label='AUC = {:.2f}'.format(auc_score))
    plt.plot([0,1],[0,1],'r--')

    plt.xlim([-0.1,1.1])
    plt.ylim([-0.1,1.1])
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')

    plt.legend(loc='lower right')
    plt.show()

### Modelo OC - Treinamento e Teste

In [ ]:
# Carregando a Base de Dados
baseObitoCurado = pd.read_excel(url)

In [ ]:
# Selecionando os Atributos do Pré-Processamento

# Antigos Atributos
#atributosSelecionados = ['faixaetaria', 'dispneia', 'qntVacinas', 'dorDeGarganta', 
#                         'coriza', 'diabetes', 'dorDeCabeca', 'cardiaca', 'evolucaoCaso']

atributosSelecionados = [ 'evolucaoCaso', 'dorDeGarganta', 'dispneia', 'coriza', 'dorDeCabeca', 
                         'diabetes', 'obesidade', 'qntVacinas', 'faixaetaria', 'faixaDiasSintomas']


baseObitoCurado = baseObitoCurado.loc[:, atributosSelecionados]

In [ ]:
baseObitoCurado[alvo].value_counts()

In [ ]:
baseObitoCurado = aplicarSmote(baseObitoCurado, alvo)
baseObitoCurado[alvo].value_counts()

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(result.drop(alvo,axis=1),
                                                    result[alvo],
                                                    test_size=0.3,
                                                    random_state=42);

In [ ]:
hiperparametrosGB = GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, 
                           random_state=1859, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [ ]:
# Fit nos dados
modeloClassificador = hiperparametrosGB
modeloClassificador.fit(x_train, y_train)

#Realizando a predição
resultadoTeste = modeloClassificador.predict(x_test)

# Verificando a acurácia
print(metrics.classification_report(y_test,resultadoTeste))

In [ ]:
# Plotando a curva ROC
exibirCurvaROC(modeloClassificador, x_test, y_test)

In [ ]:
# Exportando o modelo criado
# dump(modeloClassificador, 'modeloOC.pk1')

### Funções Utilizadas

In [ ]:
def __transformarEmPorcentagem__(valor):
    porcentagem = str(round(valor * 100, 3)) + "%"
    return porcentagem

In [ ]:
def calcularMetricas(x_test, y_test, baseDeDados, classificador, alvos):
    metricas = {}
    
    #Realizando a predição
    resultadoPredicao = classificador.predict(x_test)

    # Verificando a acurácia
    metricas['Acurácia'] = __transformarEmPorcentagem__(accuracy_score(y_test, resultadoPredicao))
    
    # Criando um dicionaŕio a partir das métricas
    metricasValores = metrics.classification_report(y_test, resultadoPredicao, output_dict=True)
    df = pd.DataFrame(metricasValores).transpose()
    
    metricas["Base"] = baseDeDados

    for classe in alvos:
        metricas["Precisão (" + str(classe) + ")"] = __transformarEmPorcentagem__(df.loc[str(classe)]['precision'])
        metricas["Recall (" + str(classe) + ")"] = __transformarEmPorcentagem__(df.loc[str(classe)]['recall'])
        metricas["F1-Score (" + str(classe) + ")"] = __transformarEmPorcentagem__(df.loc[str(classe)]['f1-score'])
    
    return metricas

### Modelo OC - Avaliando as outras Bases (Estados)

In [ ]:
listaBases = ["Vacinas/OC/oc-vacina-6040-ac.xlsx", "Vacinas/OC/oc-vacina-6040-al.xlsx", "Vacinas/OC/oc-vacina-6040-am.xlsx", "Vacinas/OC/oc-vacina-6040-ap.xlsx", "Vacinas/OC/oc-vacina-6040-ba.xlsx", 
              "Vacinas/OC/oc-vacina-6040-ce.xlsx", "Vacinas/OC/oc-vacina-6040-df.xlsx", "Vacinas/OC/oc-vacina-6040-go.xlsx", "Vacinas/OC/oc-vacina-6040-ma.xlsx", "Vacinas/OC/oc-vacina-6040-mg.xlsx", 
              "Vacinas/OC/oc-vacina-6040-mt.xlsx", "Vacinas/OC/oc-vacina-6040-pa.xlsx", "Vacinas/OC/oc-vacina-6040-pb.xlsx", "Vacinas/OC/oc-vacina-6040-pe.xlsx", "Vacinas/OC/oc-vacina-6040-pi.xlsx", 
              "Vacinas/OC/oc-vacina-6040-rj.xlsx", "Vacinas/OC/oc-vacina-6040-rn.xlsx", "Vacinas/OC/oc-vacina-6040-ro.xlsx", "Vacinas/OC/oc-vacina-6040-rr.xlsx", "Vacinas/OC/oc-vacina-6040-rs.xlsx"] 

alvo = 'evolucaoCaso'
colunasSelecionadasTeste = ['faixaetaria', 'dispneia', 'qntVacinas', 'dorDeGarganta', 
                            'coriza', 'diabetes', 'dorDeCabeca', 'cardiaca']

In [ ]:
alvos = [0, 1]
desempenhoDosModelos = pd.DataFrame()
for indice in range(0, len(listaBases)):
    covidData = pd.read_excel(listaBases[indice])
    dadosTeste = covidData.loc[:, colunasSelecionadasTeste]
    dadosAlvo =  covidData.loc[:, alvo]
     
    metricasMap = calcularMetricas(dadosTeste, dadosAlvo, listaBases[indice], modeloClassificador, alvos)
    desempenhoDosModelos = pd.concat([desempenhoDosModelos, pd.DataFrame([metricasMap])]) 

In [ ]:
desempenhoDosModelos

In [ ]:
write = pd.ExcelWriter('desempenhoModeloFinal.xlsx')
desempenhoDosModelos.to_excel(write, 'dados', index = False)
write.save()

### Modelo IC - Constatando o Desempenho

In [ ]:
# Carregando a Base de Dados
urlIC = 'Vacinas/Regioes/IC/ic-vacina-6040-sudeste.xlsx'
alvo = 'evolucaoCaso'
baseInternadoCurado = pd.read_excel(urlIC)

basesIC = {}
basesIC['Nordeste IC (60/40)'] = urlIC

In [ ]:
# Definindo o Modelo de Regressão Logística
hiperparametrosAB = AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, 
                                                           learning_rate=1.0, n_estimators=50, 
                                                           random_state=410)

hiperparametrosLR = LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True, 
                                       intercept_scaling=1, l1_ratio=None, max_iter=1000, multi_class='auto', 
                                       n_jobs=None, penalty='l2', random_state=1340, solver='lbfgs', 
                                       tol=0.0001, verbose=0, warm_start=False)


hiperparametrosGB = GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None,
                           random_state=2333, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)


modeloClassificador = {"LR": hiperparametrosLR}

In [ ]:
valoresAlvo = [1, 2]

colunasRemovidas = ['dataPrimeiraDose', 'dataSegundaDose', 'disturbiosGustatorios', 'disturbiosOlfativos', 
                    'puerpera', 'respiratoria', 'tosse', 'febre', 'assintomatico', 'obesidade']
                    
desempenhoDosModelos = mf.testarModelos(basesIC, alvo, colunasRemovidas, modeloClassificador, valoresAlvo)
desempenhoDosModelos

### Modelo IC - Treinamento e Teste

In [ ]:
# Carregando a Base de Dados
baseInternadoCurado = pd.read_excel(urlIC)
baseInternadoCurado['evolucaoCaso'].value_counts()

In [ ]:
# Convertendo os valores internado (2 -> 0)
baseInternadoCurado['evolucaoCaso'] = baseInternadoCurado['evolucaoCaso'].replace([1, 2], [1, 0])
baseInternadoCurado['evolucaoCaso'].value_counts()

In [ ]:
# Selecionando os Atributos do Pré-Processamento
baseInternadoCurado = baseInternadoCurado.drop(columns = colunasRemovidas)

In [ ]:
baseInternadoCurado

In [ ]:
alvo = 'evolucaoCaso'
baseInternadoCurado = aplicarSmote(baseInternadoCurado, alvo)

In [ ]:
baseInternadoCurado.columns

In [ ]:
# Definindo o treinamento e teste
x_train, x_test, y_train, y_test = train_test_split(baseInternadoCurado.drop(alvo,axis=1),
                                                    baseInternadoCurado[alvo],
                                                    test_size=0.3,
                                                    random_state=42);

In [ ]:
# Fit nos dados
modeloClassificador = hiperparametrosLR
modeloClassificador.fit(x_train, y_train)

#Realizando a predição
resultadoTeste = modeloClassificador.predict(x_test)

# Verificando a acurácia
print(metrics.classification_report(y_test,resultadoTeste))

In [ ]:
# Plotando a curva ROC
exibirCurvaROC(modeloClassificador, x_test, y_test)

### Modelo IC - Analisando outras Regiões

In [ ]:
# Lista das Bases de Regiões
listaBases = ["Vacinas/Regioes/IC/ic-vacina-6040-centrooeste.xlsx", "Vacinas/Regioes/IC/ic-vacina-6040-sudeste.xlsx", 
               "Vacinas/Regioes/IC/ic-vacina-6040-norte.xlsx", "Vacinas/Regioes/IC/ic-vacina-6040-nordeste.xlsx"]

# Colunas Utilizadas
colunasSelecionadasTeste = ['faixaetaria', 'sexo', 'racaCor', 'dorDeGarganta', 'dispneia', 'coriza', 
                            'dorDeCabeca', 'diabetes', 'renal', 'imunossupressao', 'fragilidadeImuno', 
                            'gestante', 'cardiaca', 'qntVacinas']

alvo = 'evolucaoCaso'

In [ ]:
alvos = [1, 2]
desempenhoDosModelos = pd.DataFrame()
for indice in range(0, len(listaBases)):
    covidData = pd.read_excel(listaBases[indice])
    dadosTeste = covidData.loc[:, colunasSelecionadasTeste]
    dadosAlvo =  covidData.loc[:, alvo]
     
    metricasMap = calcularMetricas(dadosTeste, dadosAlvo, listaBases[indice], modeloClassificador, alvos)
    desempenhoDosModelos = pd.concat([desempenhoDosModelos, pd.DataFrame([metricasMap])])
desempenhoDosModelos

In [ ]:
write = pd.ExcelWriter('desempenhoModeloICFinal.xlsx')
desempenhoDosModelos.to_excel(write, 'dados', index = False)
write.save()